In [1]:
!pip install click==8.0.3
!pip install cloudml_hypertune==0.1.0.dev6
!pip install hypertune==0.0.0
!pip uninstall matplotlib
!pip install matplotlib==3.1.3
!pip install numpy==1.20.3
!pip install pandas==1.3.4
!pip install protobuf==3.19.3
!pip install python-dotenv==0.19.2
!pip install cikit_learn==1.0.2
!pip install torch==1.10.1
!pip install transformers==4.15.0
!pip install hopsworks
!huggingface-cli lfs-enable-largefiles
!pip install wandb -qU

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 3.7 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.0.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for cloudml_hypertune: filename=cloudml_hypertune-0.1.0.dev6-py2.py3-none-any.whl size=3988 sha256=946df8a575c860c205274c1b9e33a91d904387776c7bf090266a995c86a4ed4d
  Stored in directory: /root/.cache/pip/wheels/87/fb/3b/365271726c73d8bc0b5bf39ef0f5db5a9c75

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cmdstanpy 1.0.8 requires numpy>=1.21, but you have numpy 1.20.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 92.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all t

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from huggingface_hub import login
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [4]:
import hopsworks
project = hopsworks.login()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) liangc40
	 (2) Lab1_for_iris

Enter project to access: 1

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5311


In [5]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Load Feature from Hopsworks

In [6]:
fs = project.get_feature_store()
fg = fs.get_feature_group(name="sentiment_analysis_feature_group", version=3)

Connected. Call `.close()` to terminate connection gracefully.


## Create DataLoader and TweetsDataset

In [7]:
BATCH_SIZE = 16
MAX_LEN = 160
EPOCHS = 5

In [8]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

class TweetsDataset(Dataset):
  def __init__(self, message, depression, tokenizer, max_len):
    self.message = message
    self.depression = depression
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.message)
  
  def __getitem__(self, item):
    message = str(self.message[item])
    depression = self.depression[item]

    encoding = self.tokenizer.encode_plus(
      message,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      truncation=True,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'tweet_text': message,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'depression': torch.tensor(depression, dtype=torch.long)
    }

In [9]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = TweetsDataset(
    message = df['message'].to_numpy(),
    depression = df['label'].to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size = batch_size,
    num_workers = 9
  )

In [10]:
!pip install tweet-preprocessor
import preprocessor as p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
# train_message, test_message, train_depression, test_depression = feature_view.train_test_split(0.2)

# #Creating dataloaders
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# train_data_loader = create_data_loader(train_message, train_depression, tokenizer, MAX_LEN, BATCH_SIZE)
# test_data_loader = create_data_loader(test_message, test_depression, tokenizer, MAX_LEN, BATCH_SIZE)
# data = next(iter(train_data_loader))

import pandas as pd

# df = pd.read_csv(f'/content/sentiment_tweets3.csv', encoding = 'latin-1')
# df.rename(columns = {'message to examine':'message', 'label (depression result)':'label'}, inplace = True)
# df = df.drop(columns=['Index'])
# # display(df)

# df = pd.read_csv(f'/content/twitter_training.csv', encoding = 'latin-1')
# df.columns = ['id', 'author', 'label', 'message']
# df = df[df.label != 'Irrelevant']
# df = df[df.label != 'Neutral']
# for i,v in enumerate(df['message']):
#   df.loc[i, 'message'] = p.clean(str(v))
# dic = {"Positive": int(1), "Negative": int(0)}
# df = df.replace({"label": dic})
# df = df.drop(columns=['id', 'author'])
# df = df.dropna() 

df = fg.read()

df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)
data = next(iter(train_data_loader))

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Bert-Based Depression Classier Model

In [14]:
from torch import nn, optim
import torch.nn.functional as F
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from collections import defaultdict

class DepressionClassifier(nn.Module):
  def __init__(self, n_classes, pre_trained_model_name):
    super(DepressionClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(pre_trained_model_name)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict = False #here
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [15]:
class_names = ['Not Depressed', 'Depressed']
model = DepressionClassifier(len(class_names), 'bert-base-cased')

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Training Functions

In [16]:
from torch import nn, optim
import torch.nn.functional as F
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from collections import defaultdict
import matplotlib.pyplot as plt

In [17]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    depression = d["depression"].to(device)

    outputs = model(
      input_ids = input_ids,
      attention_mask = attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, depression)

    correct_predictions += torch.sum(preds == depression)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  wandb.log({"train acc": correct_predictions.double() / n_examples, "train loss": np.mean(losses)})

  return correct_predictions.double() / n_examples, np.mean(losses)

In [18]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      depression = d["depression"].to(device)

      outputs = model(
        input_ids = input_ids,
        attention_mask = attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, depression)

      correct_predictions += torch.sum(preds == depression)
      losses.append(loss.item())

  wandb.log({"validation acc": correct_predictions.double() / n_examples, "validation loss": np.mean(losses)})

  return correct_predictions.double() / n_examples, np.mean(losses)

In [19]:
def loss_accuracy_plots(history):
    plt.figure(1)
    plt.plot(history['train_loss'])
    plt.plot(history['val_loss'])
    plt.xlabel("Epochs [-]")
    plt.ylabel("Loss [-]")
    plt.legend(['Training loss','Validation loss'])
    plt.grid()
    plt.savefig(f"/content/Training_losses_plot.jpg")
    plt.figure(2)
    plt.plot(history['train_acc'])
    plt.plot(history['val_acc'])
    plt.xlabel("Epochs [-]")
    plt.ylabel("Loss [-]")
    plt.legend(['Training accuracy','Validation accuracy'])
    plt.grid()
    plt.savefig(f"/content/Training_accuracies_plot.jpg")

## Training Data

In [20]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jan 15 08:52:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

In [23]:
print(device)

cuda:0


In [24]:
F.softmax(model(input_ids, attention_mask), dim=1)

tensor([[0.6780, 0.3220],
        [0.7192, 0.2808],
        [0.6490, 0.3510],
        [0.5992, 0.4008],
        [0.6851, 0.3149],
        [0.7139, 0.2861],
        [0.6603, 0.3397],
        [0.6739, 0.3261],
        [0.6674, 0.3326],
        [0.6194, 0.3806],
        [0.6908, 0.3092],
        [0.5658, 0.4342],
        [0.6681, 0.3319],
        [0.6862, 0.3138],
        [0.6671, 0.3329],
        [0.4990, 0.5010]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [25]:
import gc
gc.collect()

optimizer = AdamW(model.parameters(), lr = 2e-5, correct_bias = False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)

wandb.init(project="sentimental_analysis_train",
        config={"epochs": EPOCHS, "batch_size": BATCH_SIZE, "lr": 2e-5,})

loss_fn = nn.CrossEntropyLoss().to(device)
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  
  train_acc, train_loss = train_epoch(model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train))
  
  print(f'Train loss {train_loss} accuracy {train_acc}')
  
  val_acc, val_loss = eval_model(model, test_data_loader, loss_fn, device, len(df_test))
  
  print(f'Val   loss {val_loss} accuracy {val_acc}')
  
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

wandb.finish()

wandb: Currently logged in as: stan95147 (liangchen40). Use `wandb login --relogin` to force relogin


Epoch 1/5
----------
Train loss 0.697157944299622 accuracy 0.54505742471143
Val   loss 0.6254543793730286 accuracy 0.6409395973154363
Epoch 2/5
----------
Train loss 0.5419847290792271 accuracy 0.7206295021263055
Val   loss 0.48838934178383203 accuracy 0.7642906734552187
Epoch 3/5
----------
Train loss 0.3528273827545017 accuracy 0.8385165041802876
Val   loss 0.43560629136956774 accuracy 0.8147419578801204
Epoch 4/5
----------
Train loss 0.2582422596868343 accuracy 0.8841091214163798
Val   loss 0.4603640730225941 accuracy 0.8235362184679472
Epoch 5/5
----------
Train loss 0.21340634778569928 accuracy 0.9029999710706743
Val   loss 0.5040351970710012 accuracy 0.8230733626475353


train acc,▁▄▇██
train loss,█▆▃▂▁
validation acc,▁▆███
validation loss,█▃▁▂▄
train acc,0.903
train loss,0.21341
validation acc,0.82307
validation loss,0.50404


In [26]:
from google.colab import drive
drive.mount('/content/drive')
torch.save(model.state_dict(), '/content/drive/MyDrive/data/weights.pth')

Mounted at /content/drive


In [27]:
from huggingface_hub import Repository

with Repository("pytorch-model", clone_from="liangc40/sentiment_analysis", token=True).commit(commit_message="My cool model :)"):
    torch.save(model.state_dict(), "large_model.pt")

Cloning https://huggingface.co/liangc40/sentiment_analysis into local empty directory.
Pulling changes ...


Upload file large_model.pt:   0%|          | 3.30k/413M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/liangc40/sentiment_analysis
   98de0f3..98895d8  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/liangc40/sentiment_analysis
   98de0f3..98895d8  main -> main



In [28]:
from huggingface_hub import hf_hub_download

pt_file = hf_hub_download(repo_id="liangc40/sentiment_analysis", filename="large_model.pt")
model.load_state_dict(torch.load(pt_file))

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

<All keys matched successfully>

In [29]:
text = "My life is shit, gonna kill myself"
model = model.to('cpu')

encoding = tokenizer.encode_plus(text, max_length=32, add_special_tokens=True, # Add '[CLS]' and '[SEP]'
                                     return_token_type_ids=False,
                                     pad_to_max_length=True,
                                     return_attention_mask=True,
                                     return_tensors='pt')

outputs = model(input_ids = encoding['input_ids'], attention_mask = encoding['attention_mask'])
_, preds = torch.max(outputs, dim=1)
print(preds)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


tensor([1])


# Unit Testing

### Test Data

In [30]:
import unittest

In [35]:
TRAIN_SIZE = 34567
TEST_SIZE = 8642

class TestTweetsDataset(unittest.TestCase):

  def test_TweetsDataset(self):

    ds_train = TweetsDataset(
        message = df_train['message'].to_numpy(),
        depression = df_train['label'].to_numpy(),
        tokenizer=tokenizer,
        max_len=160)

    ds_test = TweetsDataset(
        message = df_test['message'].to_numpy(),
        depression = df_test['label'].to_numpy(),
        tokenizer=tokenizer,
        max_len=160)
    
    self.assertEqual(len(ds_train), TRAIN_SIZE)
    self.assertEqual(len(ds_test), TEST_SIZE)   

In [36]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_TweetsDataset (__main__.TestCalcDiv) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


In [37]:
class TestCreateDataloader(unittest.TestCase):
  def test_create_dataloader(self):
    data_loader = create_data_loader(df, tokenizer, max_len=160, batch_size=16)
    first_data =  next(iter(data_loader))

    self.assertEqual(type(first_data['input_ids']), torch.Tensor)
    self.assertEqual(first_data['input_ids'].shape, torch.Size([16, 160]))
    self.assertEqual(type(first_data['attention_mask']), torch.Tensor)
    self.assertEqual(first_data['attention_mask'].shape, torch.Size([16, 160]))

In [38]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_TweetsDataset (__main__.TestCalcDiv) ... ok
test_create_dataloader (__main__.TestCreateDataloader) ... ok

----------------------------------------------------------------------
Ran 2 tests in 1.712s

OK


### Test Model 

In [39]:
class TestModel(unittest.TestCase):

  def test_depression_classifier_init(self):
    model = DepressionClassifier(len(class_names), 'bert-base-cased')
    self.assertEqual(type(model.bert), transformers.models.bert.modeling_bert.BertModel)
    self.assertEqual(type(model.drop), nn.Dropout)
    self.assertEqual(type(model.out), nn.Linear)

In [40]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_TweetsDataset (__main__.TestCalcDiv) ... ok
test_create_dataloader (__main__.TestCreateDataloader) ... ok
test_depression_classifier_init (__main__.TestModel) ... Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificati

In [42]:
MAX_LEN = 160
BATCH_SIZE = 16

data_loader = create_data_loader(df_train, tokenizer, max_len=MAX_LEN, batch_size=BATCH_SIZE)
first_data =  next(iter(data_loader))
input_ids = first_data['input_ids']
attention_mask = first_data['attention_mask']

class TestModel(unittest.TestCase):

  def test_depression_classifier_forward(self):
    model = DepressionClassifier(len(class_names), 'bert-base-cased')
    out = model(input_ids, attention_mask)
    self.assertEqual(first_data['input_ids'].shape, torch.Size([BATCH_SIZE, MAX_LEN]))
    self.assertEqual(first_data['attention_mask'].shape, torch.Size([BATCH_SIZE, MAX_LEN]))
    self.assertEqual(out.ndim, 2)
    self.assertEqual(out.shape, torch.Size([BATCH_SIZE, 2]))

In [43]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_TweetsDataset (__main__.TestCalcDiv) ... ok
test_create_dataloader (__main__.TestCreateDataloader) ... ok
test_depression_classifier_forward (__main__.TestModel) ... Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassific